In [1]:
# load the dataset
base_dir = r"D:\2023springcourses\CS6665\cs6665 course project\data"
for dirname, _, filenames in os.walk(base_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

NameError: name 'os' is not defined

In [ ]:
##import libraries
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import xgboost as xgb
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from scipy import stats
#import jpx_tokyo_market_prediction
import warnings; warnings.filterwarnings("ignore")

## 1. Understand how to calculate scores and deepen your understanding of Rank ##
https://www.kaggle.com/code/smeitoma/jpx-competition-metric-definition
https://www.kaggle.com/code/chumajin/easy-to-understand-the-competition?scriptVersionId=94143164

In [ ]:
# look for the stock_prices.csv in train file
stock_prices=pd.read_csv(f"{base_dir}/train_files"+'/stock_prices.csv')
stock_prices

In [ ]:
stock_prices["Date"] = pd.to_datetime(stock_prices["Date"])

In [ ]:
# look for only one stock,eg.1301
tmpdf = stock_prices[stock_prices["SecuritiesCode"]==1301].reset_index(drop=True)
tmpdf.head(3)

In [ ]:
 # calculate TARGET (change rate of Close on the next day and the next day) by yourself.
tmpdf["Close_shift1"] = tmpdf["Close"].shift(-1)
tmpdf["Close_shift2"] = tmpdf["Close"].shift(-2)

tmpdf["rate"] = (tmpdf["Close_shift2"] - tmpdf["Close_shift1"]) / tmpdf["Close_shift1"]
tmpdf

the Target and the rate calculated match. (so I will use the Target that is calculated from now on)

## Rank calculation. ##
First, let's take a look at only one day. * Please note that not all 2000 stocks have data depending on the date.

In [ ]:
tmpdf2 = stock_prices[stock_prices["Date"]=="2021-12-03"].reset_index(drop=True)
tmpdf2

Rank in descending order of Target. I do sort it for understanding. Rank is tied to 0-1999, so don't forget -1 

In [ ]:
tmpdf2["rank"] = tmpdf2["Target"].rank(ascending=False,method="first") -1 
tmpdf2 = tmpdf2.sort_values("rank").reset_index(drop=True)
tmpdf2

## Calculation of daily spread return of this day ##



In [ ]:
tmpdf2_top200 = tmpdf2.iloc[:200,:]
weights = np.linspace(start=2, stop=1, num=200)
tmpdf2_top200["weights"] = weights
tmpdf2_top200["calc_weights"] = tmpdf2_top200["Target"] * tmpdf2_top200["weights"]
Sup = tmpdf2_top200["calc_weights"].sum()/np.mean(weights)
Sup

In [ ]:
tmpdf2_bottom200 = tmpdf2.iloc[-200:,:]
tmpdf2_bottom200 = tmpdf2_bottom200.sort_values("rank",ascending = False).reset_index(drop=True)
tmpdf2_bottom200["weights"] = weights
tmpdf2_bottom200["calc_weights"] = tmpdf2_bottom200["Target"] * tmpdf2_bottom200["weights"]
Sdown = tmpdf2_bottom200["calc_weights"].sum()/np.mean(weights)
Sdown


In [ ]:
# calculate daily spread return
daily_spread_return = Sup - Sdown
daily_spread_return

In [ ]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [ ]:
# calculate which days have 2000 stocks
idcount = stock_prices.groupby("Date")["SecuritiesCode"].count().reset_index()
idcount

In [ ]:
plt.plot(idcount["Date"],idcount["SecuritiesCode"])

In [ ]:
idcount.loc[idcount["SecuritiesCode"]==2000,:]

In [ ]:
stock_prices2 = stock_prices.loc[stock_prices["Date"]>= "2021-01-01"].reset_index(drop=True)
stock_prices2

In [ ]:
stock_prices2["Rank"] = stock_prices2.groupby("Date")["Target"].rank(ascending=False,method="first") -1 
stock_prices2["Rank"] =stock_prices2["Rank"].astype("int")
stock_prices2["Rank"].min()
score = calc_spread_return_sharpe(stock_prices2, portfolio_size= 200, toprank_weight_ratio= 2)
score

In [ ]:
supplemental_stock_prices=pd.read_csv(f"{base_dir}/supplemental_files"+'/stock_prices.csv')
supplemental_stock_prices["Rank"] = supplemental_stock_prices.groupby("Date")["Target"].rank(ascending=False,method="first") -1
idcount1 = supplemental_stock_prices.groupby("Date")["SecuritiesCode"].count().reset_index()
idcount1.loc[idcount1["SecuritiesCode"]==2000,:]



In [ ]:
finday = supplemental_stock_prices[supplemental_stock_prices["Date"]=="2022-04-22"].reset_index(drop=True)
finday

In [ ]:
finday[finday["Rank"]==finday["Rank"].iloc[0]]
#finday["Rank"] = finday["Rank"].astype("int")
findaydict = dict(zip(finday["SecuritiesCode"],finday["Rank"]))

In [ ]:
#sample_prediction["Rank"]  = sample_prediction["SecuritiesCode"].map(findaydict)
#sample_prediction

In [ ]:
stock_prices=pd.read_csv(f"{base_dir}/train_files"+'/stock_prices.csv')
secondary_stock_prices=pd.read_csv(f"{base_dir}/train_files"+'/secondary_stock_prices.csv')
supplemental_stock_prices=pd.read_csv(f"{base_dir}/supplemental_files"+'/stock_prices.csv')
supplemental_secondary_stock_prices=pd.read_csv(f"{base_dir}/supplemental_files"+'/secondary_stock_prices.csv')

In [ ]:
stock_prices.head(-5)

In [ ]:
secondary_stock_prices.head(-5)

In [ ]:
supplemental_stock_prices.head(-5)

In [ ]:
supplemental_secondary_stock_prices.head(-5)

In [ ]:
# join the datas in the train file and supplemental files
stock_prices = stock_prices.append(secondary_stock_prices)
stock_prices = stock_prices.append(supplemental_stock_prices)
stock_prices= stock_prices.append(supplemental_secondary_stock_prices)
stock_prices

In [ ]:
stock_prices.head(-5)

## Features Analysis ##

In [ ]:
# Convert 'Date' column to datetime type
stock_prices['Date'] = pd.to_datetime(stock_prices['Date'])

# Get the data in the year 2021 
stock_prices2021 = stock_prices[stock_prices['Date'] > pd.to_datetime('2021-01-01')]

stock_prices2021=stock_prices2021.drop(columns=['RowId','Date','SecuritiesCode','ExpectedDividend','SupervisionFlag'])

stock_prices2021

In [ ]:
# calculate the correlations about features
sns.heatmap(stock_prices2021.corr(), annot=True)
plt.show()

In [ ]:
g = sns.PairGrid(data = stock_prices2021, vars = ['Open', 'High', 'Low','Close','Volume','Target'])
g.map_offdiag(plt.scatter)

1. from the graphs we can see the features(Open,High,Low,Close) are postive correlation.
2. The relationshio between the features(Open,High,Low,Close,Volume) and target presents a normal distribution.

1. from the stock_prices.cvs, we can know that the values of ExpectedDividend are nan, so we will set  the value to zero.
2. the values of SupervisionFlag is binary, so we change them to int.
3. if some values of target are missing, we will set 0 to them.
4.we will choose the features:'Open', 'High', 'Low', 'Close'. becase In investing and trading, Z-scores are measures of an instrument's variability and can be used by traders to help determine volatility. so we will use zscore to scale the values.

In [ ]:
def feature_engineering(data):
    data['ExpectedDividend'] = data['ExpectedDividend'].fillna(0)
    data["SupervisionFlag"] = data["SupervisionFlag"].astype(int)
    data['Target'] = data['Target'].fillna(0)
    
    cols = ['Open', 'High', 'Low', 'Close']
    data.loc[:,cols] = data.loc[:,cols].ffill()
    data.loc[:,cols] = data.loc[:,cols].bfill()

    

    data['Open'] = stats.zscore(data['Open'])
    data['High'] = stats.zscore(data['High'])
    data['Low'] = stats.zscore(data['Low'])
    data['Close'] = stats.zscore(data['Close'])
   
    
    data = data.drop(['RowId', 'Date'], axis=1)
    
    return data

In [ ]:
def featuring_2(data):
    data['ExpectedDividend'] = data['ExpectedDividend'].fillna(0)
    data["SupervisionFlag"] = data["SupervisionFlag"].astype(int)
    
    cols = ['Open', 'High', 'Low', 'Close']
    data.loc[:,cols] = data.loc[:,cols].ffill()
    data.loc[:,cols] = data.loc[:,cols].bfill()

    
    
    data['Open'] = stats.zscore(data['Open'])
    data['High'] = stats.zscore(data['High'])
    data['Low'] = stats.zscore(data['Low'])
    data['Close'] = stats.zscore(data['Close'])
   
    
    data = data.drop(['RowId', 'Date','Target'], axis=1)
    
    return data

In [ ]:
df=feature_engineering(stock_prices)
df

In [ ]:
df_1=df.drop(['Target'],axis=1)
df_1.shape


## create models ##

generate train set and test set

In [ ]:
df_X=df.drop(['Target'], axis=1)
df_Y=df['Target']

In [ ]:
df_X.head()

In [ ]:
df_Y.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_X,df_Y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape,X_test.shape

## model 1 : Linear regression ##

In [ ]:
from sklearn.linear_model import LinearRegression

# Create an instance of the LinearRegression class
model_reg = LinearRegression()

# Train the model on the training data
model_reg.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model_reg.predict(X_test)

# print the parameters
print(model_reg.coef_) 
print(model_reg.intercept_) 
print(model_reg.get_params())

In [ ]:
print(y_pred,y_test)

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
import math
print(math.sqrt(mean_squared_error(y_pred,y_test)))

In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

# Generate the learning curve
train_sizes, train_scores, test_scores = learning_curve(model_reg, X_train, y_train, cv=5)

# Calculate the mean and standard deviation of the training scores and testing scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.plot(train_sizes, train_mean, label='Train')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.plot(train_sizes, test_mean, label='Test')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.xlabel('Number of training sizes')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend()
plt.show()

In [ ]:
def calc_spread_return_per_day(df, portfolio_size=200, toprank_weight_ratio=2):
    assert df['Rank'].min() == 0
    assert df['Rank'].max() == len(df['Rank']) - 1
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    return purchase - short

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size=200, toprank_weight_ratio=2):
    buf = df.groupby('Date').apply(calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio#, buf

def add_rank(df):
    df["Rank"] = df.groupby("Date")["Target"].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

def fill_nan_inf(df):
    df = df.fillna(0)
    df = df.replace([np.inf, -np.inf], 0)
    return df

def check_score(df,preds,Securities_filter=[]):
    tmp_preds=df[['Date','SecuritiesCode']].copy()
    tmp_preds['Target']=preds
    
    #Rank Filter. Calculate median for this date and assign this value to the list of Securities to filter.
    tmp_preds['target_mean']=tmp_preds.groupby("Date")["Target"].transform('median')
    tmp_preds.loc[tmp_preds['SecuritiesCode'].isin(Securities_filter),'Target']=tmp_preds['target_mean']
    
    tmp_preds = add_rank(tmp_preds)
    df['Rank']=tmp_preds['Rank']
    score=round(calc_spread_return_sharpe(df, portfolio_size= 200, toprank_weight_ratio= 2),5)
    score_mean=round(df.groupby('Date').apply(calc_spread_return_per_day, 200, 2).mean(),5)
    score_std=round(df.groupby('Date').apply(calc_spread_return_per_day, 200, 2).std(),5)
    print(f'Competition_Score:{score}, rank_score_mean:{score_mean}, rank_score_std:{score_std}')
    

In [ ]:
check_score(test,y_pred)